# Парсер параметров в тексте и преобразование в числовой вид

### Установка Yargy

In [14]:
#!pip install yargy

# Импорт

In [2]:
from yargy.interpretation import fact
from yargy import (
    Parser,
    rule
)
from yargy.predicates import (
    normalized,
    dictionary
)
from yargy.pipelines import morph_pipeline
import re

### Текст с которым будем работать

In [8]:
text = """vдобрый день представьтесь пожалуйста меня зовут иван иванов я представляю компанию ооо триал согласно соглашению номер восемьсот пятьдесят семь пятого марта две тысячи двадцатого года дата сделки сегодня первое января две тысячи двадцать второго года начала действия сделки сегодня окончание действия сделки первого марта две тысячи двадцать пятого года на сумму один миллион рублей"""

### Предподготовка текста и поиск фрагмента 

In [9]:
def preprocess(text):
    text = text.replace('\n', ' ')
    text = find_section(text)
    return text

def find_section(text):
    res = []
    result = re.finditer('соглашени[еяю]\s', text)
    for el in result:
        res.append(text[el.start()-10:el.end()+45])
    return res 

line = preprocess(text)

### Теперь создаем правила для Yargy

In [10]:
Number = fact(
    'number',
    ['num1', 'num2', 'num3', 'flag']
)

DIGIT_NUM1 = {
    'один': 1,
    'два': 2,
    'три': 3,
    'четыре': 4,
    'пять': 5,
    'шесть': 6,
    'семь': 7,
    'восемь': 8,
    'девять': 9,
    'сто': 100,
    'двести': 200,
    'триста': 300,
    'четыреста': 400,
    'пятьсот': 500,
    'шестьсот': 600,
    'семьсот': 700,
    'восемьсот': 800,
    'девятьсот': 900
}

DIGIT_NAME_NUM1 = dictionary(
    DIGIT_NUM1
)

DIGIT_NUM2 = {
    'один': 1,
    'два': 2,
    'три': 3,
    'четыре': 4,
    'пять': 5,
    'шесть': 6,
    'семь': 7,
    'восемь': 8,
    'девять': 9,
    'десять': 10,
    'одиннадцать': 11,
    'двенадцать': 12,
    'тринадцать': 13,
    'четырнадцать': 14,
    'пятнадцать': 15,
    'шестнадцать': 16,
    'семнадцать': 17,
    'восемнадцать': 18,
    'девятнадцать': 19,
    'двадцать': 20,
    'тридцать': 30,
    'сорок': 40,
    'пятьдесят': 50,
    'шестьдесят': 60,
    'семьдесят': 70,
    'восемьдесят': 80,
    'девяносто': 90
}

DIGIT_NAME_NUM2 = dictionary(
    DIGIT_NUM2
)

DIGIT_NUM3 = {
    'один': 1,
    'два': 2,
    'три': 3,
    'четыре': 4,
    'пять': 5,
    'шесть': 6,
    'семь': 7,
    'восемь': 8,
    'девять': 9
}

DIGIT_NAME_NUM3 = dictionary(
    DIGIT_NUM3
)

MONTH_NAME = morph_pipeline([
    'январь',
    'февраль',
    'март',
    'апрель',
    'май',
    'июнь',
    'июль',
    'август',
    'сентябрь',
    'октябрь',
    'ноябрь',
    'декабрь'
]).interpretation(Number.flag)

RULE = rule(
    normalized('соглашение'),
    normalized('номер').optional(),
    DIGIT_NAME_NUM1.interpretation(Number.num1.normalized().custom(DIGIT_NUM1.get)),
    DIGIT_NAME_NUM2.interpretation(Number.num2.normalized().custom(DIGIT_NUM2.get)).optional(),
    DIGIT_NAME_NUM3.interpretation(Number.num3.normalized().custom(DIGIT_NUM3.get)).optional(),
    MONTH_NAME.optional()
).interpretation(
    Number
)


### Токены

In [11]:
parser = Parser(RULE)
matches = list(parser.findall(text))
matches

[Match(
     tokens=[MorphToken(
          value='соглашению',
          span=[103, 113),
          type='RU',
          forms=[Form('соглашение', Grams(NOUN,datv,inan,neut,sing))]
      ),
      MorphToken(
          value='номер',
          span=[114, 119),
          type='RU',
          forms=[Form('номер', Grams(NOUN,inan,masc,nomn,sing)),
           Form('номер', Grams(NOUN,accs,inan,masc,sing))]
      ),
      MorphToken(
          value='восемьсот',
          span=[120, 129),
          type='RU',
          forms=[Form('восемьсот', Grams(NUMR,nomn)),
           Form('восемьсот', Grams(NUMR,accs))]
      ),
      MorphToken(
          value='пятьдесят',
          span=[130, 139),
          type='RU',
          forms=[Form('пятьдесят', Grams(NUMR,nomn)),
           Form('пятьдесят', Grams(NUMR,accs))]
      ),
      MorphToken(
          value='семь',
          span=[140, 144),
          type='RU',
          forms=[Form('семь', Grams(NUMR,nomn)),
           Form('семь', Grams(NUMR,

### Значения которые извлеклись и преобразовались

In [23]:
facts = [_.fact for _ in matches]

facts

[number(
     num1=800,
     num2=50,
     num3=7,
     flag=None
 )]

### Извлечем и преобразуем числа

In [24]:
def extractor(text):
    fraction = preprocess(text)
    for el in fraction:
        matches = parser.findall(el)
        if matches:
            result = []
            res = []
            facts = [_.fact for _ in matches]
            for f in facts:
                res += [f.num1, f.num2, f.num3]
                res = [int(i) for i in res if i]
                if len(res) == 1:
                    result.append(f.num1)
                elif len(res) == 2:
                    if f.flag:
                        result.append(res[0])
                    else:
                        result.append(f"{res[0]+res[1]}")
                elif len(res)==3 and res[0]<10 and res[1]<10 and res[2]<10 and not f.flag:
                    digit = f"{res[0]}{res[1]}{res[2]}"
                    result.append(int(digit))
                elif len(res)==3:
                    if f.flag:
                        result.append(f"{res[0]+res[1]}")
                    else:
                        result.append(f"{res[0]+res[1]+res[2]}")
            return ''.join(result)
        else:
            return ''
     
        
extractor(text)

'857'